In [1]:
!pip install streamlit pyngrok pandas numpy plotly scikit-learn


In [11]:
%%writefile app.py
import streamlit as st
import pandas as pd
import numpy as np
import plotly.express as px
from sklearn.ensemble import IsolationForest

# ---------------- PAGE CONFIG ----------------
st.set_page_config(page_title="Health Anomaly Dashboard", layout="wide")

st.title("🏥 Health Anomaly Detection Dashboard")
st.markdown("Upload fitness data (CSV) to detect anomalies")

# ---------------- FILE UPLOAD ----------------
uploaded_file = st.file_uploader("Upload CSV file", type=["csv"])

# ---------------- FUNCTIONS ----------------
def preprocess_fitness_data(df):
    df = df.copy()

    # Clean column names
    df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")

    # Auto-parse timestamp (SAFE)
    df["timestamp"] = pd.to_datetime(df["timestamp"], errors="coerce", dayfirst=True)

    # Drop rows with invalid timestamp
    df = df.dropna(subset=["timestamp"])

    # Sort by time
    df = df.sort_values("timestamp")

    return df

def detect_anomalies(df, metric):
    # Safety checks
    if df.empty:
        st.error("❌ No valid timestamp data found after preprocessing.")
        st.stop()

    if df.shape[0] < 3:
        st.warning("⚠️ Dataset too small for anomaly detection (need ≥ 3 rows).")
        df["anomaly"] = "Normal"
        return df

    model = IsolationForest(
        contamination=min(0.2, 1 / len(df)),
        random_state=42
    )

    X = df[[metric]].fillna(df[metric].median())

    df["anomaly"] = model.fit_predict(X)
    df["anomaly"] = df["anomaly"].map({1: "Normal", -1: "Anomaly"})

    return df

# ---------------- MAIN LOGIC ----------------
if uploaded_file:
    df_raw = pd.read_csv(uploaded_file)

    st.subheader("📄 Raw Data Preview")
    st.dataframe(df_raw)

    df = preprocess_fitness_data(df_raw)

    numeric_metrics = df.select_dtypes(include=np.number).columns.tolist()

    if not numeric_metrics:
        st.error("❌ No numeric columns found for analysis.")
        st.stop()

    st.sidebar.header("⚙️ Controls")
    metric = st.sidebar.selectbox("Select Metric", numeric_metrics)

    df = detect_anomalies(df, metric)

    # ---------------- PLOT ----------------
    st.subheader(f"📈 {metric.replace('_',' ').title()} Trend")

    fig = px.line(
        df,
        x="timestamp",
        y=metric,
        color="anomaly",
        markers=True,
        title=f"{metric.replace('_',' ').title()} with Anomalies"
    )

    st.plotly_chart(fig, use_container_width=True)

    # ---------------- ANOMALY TABLE ----------------
    st.subheader("🚨 Detected Anomalies")
    st.dataframe(df[df["anomaly"] == "Anomaly"])

    # ---------------- DOWNLOAD ----------------
    csv = df.to_csv(index=False).encode("utf-8")
    st.download_button(
        "⬇️ Download Result CSV",
        csv,
        "anomaly_output.csv",
        "text/csv"
    )

else:
    st.info("⬆️ Please upload a CSV file to continue")


Overwriting app.py


In [12]:
from pyngrok import ngrok

ngrok.set_auth_token("36byQbgfeXjiJqt5Dk7WpO9fzoy_4NTSEZSZz44vXtK6Zk2oC")


In [13]:
!streamlit run app.py &>/dev/null &


In [14]:
public_url = ngrok.connect(8501)
public_url


<NgrokTunnel: "https://giancarlo-dedicatory-mauricio.ngrok-free.dev" -> "http://localhost:8501">